In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.plasma.standard_plasmas import assemble_plasma
from tardis.model import Radial1DModel
from tardis.io.atom_data import AtomData
from tardis.analysis import LastLineInteraction
from tardis.util.base import int_to_roman
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tardis.io.config_internal import get_data_dir
from astropy import units as u
import plotly.io as pio
pio.renderers.default='notebook'
import os

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
### Run Simulation
config = Configuration.from_yaml('../../../docs/tardis_example.yml')
atom_data = AtomData.from_hdf(os.path.join(get_data_dir(), 'kurucz_cd23_chianti_H_He.h5'))
model = Radial1DModel.from_config(config, atom_data=atom_data)
plasma = assemble_plasma(config, model, atom_data=atom_data)
sim = Simulation.from_config(config, model=model, plasma=plasma)
sim.run()

Abundances have not been normalized to 1. - normalizing
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:   ', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

Shell No.,t_rad,next_t_rad,w,next_w
0,9.93e+03,1.01e+04,0.4,0.525
5,9.85e+03,1.03e+04,0.211,0.196
10,9.78e+03,1.02e+04,0.143,0.115
15,9.71e+03,9.88e+03,0.105,0.0843


/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



...

Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.48,0.486
5,1.16e+04,1.15e+04,0.174,0.18
10,1.13e+04,1.12e+04,0.105,0.108
15,1.09e+04,1.09e+04,0.0789,0.0793


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.486
5,1.15e+04,1.15e+04,0.18,0.177
10,1.12e+04,1.13e+04,0.108,0.107
15,1.09e+04,1.09e+04,0.0793,0.0811


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.483
5,1.15e+04,1.16e+04,0.177,0.17
10,1.13e+04,1.13e+04,0.107,0.107
15,1.09e+04,1.09e+04,0.0811,0.0799


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.483,0.482
5,1.16e+04,1.16e+04,0.17,0.172
10,1.13e+04,1.13e+04,0.107,0.105
15,1.09e+04,1.09e+04,0.0799,0.0807


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.482,0.478
5,1.16e+04,1.14e+04,0.172,0.177
10,1.13e+04,1.13e+04,0.105,0.107
15,1.09e+04,1.08e+04,0.0807,0.0814


In [3]:
### Define constants (Ion and number of shells to display)
ATOMIC_NUMBER = 2 # He
ION_NUMBER = 0
ATOM_NAME = sim.plasma.atomic_data.atom_data.loc[ATOMIC_NUMBER]['name']
ATOMIC_SYMBOL = sim.plasma.atomic_data.atom_data.loc[ATOMIC_NUMBER]['symbol']

In [4]:
NUM_SHELLS = 10 # Should be input by user (or dependent on element)

### Get energy levels and convert to eV
raw_energy_levels = sim.plasma.atomic_data.levels.loc[ATOMIC_NUMBER, ION_NUMBER].loc[1:NUM_SHELLS]
raw_energy_levels = raw_energy_levels.energy / (u.electronvolt.to('J') * 1e7)
raw_energy_levels = pd.Series(raw_energy_levels, name='energy')

In [5]:
### Get level populations
raw_level_populations = sim.plasma.level_number_density.loc[ATOMIC_NUMBER, ION_NUMBER].loc[1:NUM_SHELLS]

# Average out the level populations across all zones (TODO: might include an option to select the zone number later)
raw_level_populations = raw_level_populations.mean(axis=1)

raw_level_populations = pd.Series(raw_level_populations, name='population')

In [6]:
### Join level populations and energy values
raw_level_data = pd.merge(raw_energy_levels, raw_level_populations, left_index=True, right_index=True)

In [7]:
### Merge the levels if energy difference is too less
LEVEL_DIFF_THRESHOLD = 0.01

# Get new level numbers
raw_level_data['merged_level_number'] = (raw_level_data['energy'].pct_change().abs() > LEVEL_DIFF_THRESHOLD).cumsum() + 1

# Group data with new level numbers
level_data = raw_level_data.reset_index().groupby('merged_level_number').agg(
                                 energy=('energy', 'mean'), # Set energy as mean of merged levels
                                 population=('population', 'sum')) # Add the populations of merged levels

In [8]:
FILTER_MODE = 'packet_out_nu' # Emitted option

### Get lines data
lines_data = sim.plasma.atomic_data.lines.loc[ATOMIC_NUMBER, ION_NUMBER].reset_index().set_index('line_id')

# Get relevant lines for current simulation
last_line_interaction = LastLineInteraction.from_model(sim, FILTER_MODE)
last_line_interaction.atomic_number = ATOMIC_NUMBER
last_line_interaction.ion_number = ION_NUMBER

/var/folders/y3/003r_9sn0ysgqcll3s5phtyc0000gn/T/ipykernel_49824/634769751.py:4: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [9]:
# Get the excitation/de-excitation transitions from last_line_interaction object
excit_lines = last_line_interaction.last_line_in.reset_index().groupby(['level_number_lower', 'level_number_upper']).agg(
    num_electrons=('line_id', 'count'), # Take count of lines
    wavelength=('wavelength', 'first'), # Take first of wavelengths
).reset_index()

deexcit_lines = last_line_interaction.last_line_out.reset_index().groupby(['level_number_lower', 'level_number_upper']).agg(
    num_electrons=('line_id', 'count'), # Take count of lines
    wavelength=('wavelength', 'first'), # Take first of wavelengths
).reset_index()

# Filter transitions to only include transitions upto the NUM_SHELLS
excit_lines = excit_lines.loc[
    (excit_lines.level_number_lower <= NUM_SHELLS) & (excit_lines.level_number_upper <= NUM_SHELLS)
]
deexcit_lines = deexcit_lines.loc[
    (deexcit_lines.level_number_lower <= NUM_SHELLS) & (deexcit_lines.level_number_upper <= NUM_SHELLS)
]

# Map the levels to merged levels
excit_lines['merged_level_number_lower'] = excit_lines.level_number_lower.map(raw_level_data.merged_level_number)
excit_lines['merged_level_number_upper'] = excit_lines.level_number_upper.map(raw_level_data.merged_level_number)
deexcit_lines['merged_level_number_lower'] = deexcit_lines.level_number_lower.map(raw_level_data.merged_level_number)
deexcit_lines['merged_level_number_upper'] = deexcit_lines.level_number_upper.map(raw_level_data.merged_level_number)

# Group by level pairs
excit_lines = excit_lines.groupby(['merged_level_number_lower', 'merged_level_number_upper']).agg(
    wavelength=('wavelength', 'mean'), # Take mean of wavelength
    num_electrons=('num_electrons', 'sum'), # Take sum of counts
).reset_index()
deexcit_lines = deexcit_lines.groupby(['merged_level_number_lower', 'merged_level_number_upper']).agg(
    wavelength=('wavelength', 'mean'), # Take mean of wavelength
    num_electrons=('num_electrons', 'sum'), # Take sum of counts
).reset_index()

# Compute the standardized log number of electrons for transition line thickness and offset
excit_log_num_electrons_range = np.log10(np.max(excit_lines.num_electrons)) - np.log10(np.min(excit_lines.num_electrons))
excit_lines['standard_log_num_electrons'] = (np.log10(excit_lines.num_electrons) - np.log10(np.min(excit_lines.num_electrons))) / excit_log_num_electrons_range

deexcit_log_num_electrons_range = np.log10(np.max(deexcit_lines.num_electrons)) - np.log10(np.min(deexcit_lines.num_electrons))
deexcit_lines['standard_log_num_electrons'] = (np.log10(deexcit_lines.num_electrons) - np.log10(np.min(deexcit_lines.num_electrons))) / deexcit_log_num_electrons_range

In [15]:
MERGED_NUM_SHELLS = level_data.index.max()

### Create figure and set metadata
fig = go.Figure()

fig.update_layout(
    title=f"Grotrian Diagram for {ATOM_NAME} {int_to_roman(ION_NUMBER)}",
    title_x=0.5,
    yaxis_title="Energy Level (eV)",
    plot_bgcolor='white',
    autosize=False,
    width=700,
    height=700,
    xaxis=dict(showticklabels=False, showgrid=False,),  # Hide x-axis labels
    yaxis=dict(showgrid=False,),  # Adjust y-axis range accordingly,
    showlegend=False,
)

### Create energy level platforms in the figure
LEVEL_WIDTH_SCALE = 3
LEVEL_WIDTH_MIN = 1
level_data['centered_log_population'] = np.log10(level_data.population) - np.log10(np.min(level_data.population))
for level_number, level_info in level_data.iterrows():
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=[level_info.energy, level_info.energy],
        mode='lines+text',
        text=['', f'n={level_number}'], textposition='top right',
        line=dict(color='black', width=level_info.centered_log_population * LEVEL_WIDTH_SCALE + LEVEL_WIDTH_MIN),
        showlegend=False))

### Create transition lines
# Define colors for the transitions based on wavelengths
cmap = plt.get_cmap('plasma')
min_wavelength = np.log10(sim.plasma.lines.wavelength.min())
max_wavelength = np.log10(sim.plasma.lines.wavelength.max())
wavelength_range = max_wavelength - min_wavelength

# Transition offset for avoiding overlap of transitions
TRANSITION_OFFSET = 0.1  # Horizontal offset for transitions
SLANT = 0.01  # Horizontal offset for slanting

WIDTH_SCALE = 2
MIN_WIDTH = 1

ARROW_WIDTH_SCALE = 1
ARROW_MIN_WIDTH = 0.1

X_MAX = 1.35

# Plot excitation transitions
for _, line_info in excit_lines.iterrows():
    lower, upper = line_info.merged_level_number_lower, line_info.merged_level_number_upper
    wavelength, num_electrons = line_info.wavelength, line_info.standard_log_num_electrons
    energy_lower, energy_upper = level_data.loc[lower].energy, level_data.loc[upper].energy
    
    x_end = (upper - lower) / (MERGED_NUM_SHELLS - lower) * X_MAX
    
    color = matplotlib.colors.rgb2hex(cmap((np.log10(wavelength)-min_wavelength) / wavelength_range)[:3])

    # Add arrowhead
    fig.add_annotation(
        x=x_end, y=energy_upper,  # Start of arrow
        ax=0, ay=energy_lower,  # End of arrow
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowcolor=color,
        arrowhead=2,  # Arrow style
        arrowsize=1,  # Make the arrowhead 2 times bigger
        arrowwidth=num_electrons * WIDTH_SCALE + MIN_WIDTH,  # Adjust width accordingly
    )

# Plot deexcitation transitions
for _, line_info in deexcit_lines.iterrows():
    lower, upper = line_info.merged_level_number_lower, line_info.merged_level_number_upper
    wavelength, num_electrons = line_info.wavelength, line_info.standard_log_num_electrons
    energy_lower, energy_upper = level_data.loc[lower].energy, level_data.loc[upper].energy
    
    x_end = (upper - lower) / (MERGED_NUM_SHELLS - lower) * X_MAX
    
    color = matplotlib.colors.rgb2hex(cmap((np.log10(wavelength)-min_wavelength) / wavelength_range)[:3])

    # Add arrowhead
    fig.add_annotation(
        x=x_end, y=energy_lower,  # Start of arrow
        ax=0, ay=energy_upper,  # End of arrow
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowcolor=color,
        arrowhead=2,  # Arrow style
        arrowsize=1,  # Make the arrowhead 2 times bigger
        arrowwidth=num_electrons * WIDTH_SCALE + MIN_WIDTH,  # Adjust width accordingly
    )

fig.show()

/var/folders/y3/003r_9sn0ysgqcll3s5phtyc0000gn/T/ipykernel_49824/2594994284.py:57: RuntimeWarning:

invalid value encountered in double_scalars

/var/folders/y3/003r_9sn0ysgqcll3s5phtyc0000gn/T/ipykernel_49824/2594994284.py:80: RuntimeWarning:

invalid value encountered in double_scalars

